# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2


Group Number: 13

Group Members:
- Jonathan Wiguna (18222019)
- Naomi Pricilla Agustine(18222065)
- Harry Truman Suhalim (18222081)
- Micky Valentino(18222093)

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.utils.validation import check_is_fitted, check_array
from imblearn.combine import SMOTETomek
from datetime import datetime
import re

## Import Dataset

In [5]:
# Import library yang diperlukan
import pandas as pd

# Baca file TSV
df = pd.read_csv('uniprotkb_AND_reviewed_true_AND_existen_2025_04_22.tsv', sep='\t')

# Tampilkan 5 baris pertama untuk memeriksa data
df.head(5)

,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence,Reviewed,Helix,Turn,Beta strand,PubMed ID,DOI ID
0,A0A068B6Q6,CA18_CONBE,Conotoxin Bt1.8,NaN,Conus betulinus (Beech cone),37,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,reviewed,"HELIX 28..31; /evidence=""ECO:0007829|PDB:2NAY""",NaN,"STRAND 23..26; /evidence=""ECO:0007829|PDB:2NAY""",34008858,10.1111/jnc.15434
1,A0A088MIT0,BRKP2_PHYNA,Bradykinin-related peptides [Cleaved into: Nat...,BBN,Physalaemus nattereri (Cuyaba dwarf frog) (Eup...,134,MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...,reviewed,NaN,NaN,NaN,26443407,10.1002/rcm.7313
2,A0A097PTA8,DEFCO_COPCI,Fungal defensin copsin,NaN,Coprinopsis cinerea (Inky cap fungus) (Hormogr...,184,MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...,reviewed,NaN,NaN,NaN,25342741; 28825809,10.1074/jbc.m114.599878; 10.1021/acs.biochem.7...
3,A0A0A0LLY1,SRBP1_CUCSA,Small RNA binding protein 1 (CsSRBP1),SRBP1 Csa_2G247040,Cucumis sativus (Cucumber),160,MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...,reviewed,NaN,NaN,NaN,31812689; 19881527,10.1016/j.molp.2019.12.001; 10.1038/ng.475
4,A0A0A1I6E7,NDB4S_ANDCR,Antimicrobial peptide AcrAP1,NaN,Androctonus crassicauda (Arabian fat-tailed sc...,74,MEIKYLLTVFLVLLIVSDHCQAFLFSLIPHAISGLISAFKGRRKRD...,reviewed,NaN,NaN,NaN,25332684,10.7150/ijbs.9859


In [ ]:
# prompt: print seluruh isi Helix dari data dengan Entry 'O00168'

print(df[df['Entry'] == 'O00168']['Helix'])

356    HELIX 23..26; /evidence="ECO:0007829|PDB:2JO1"...
Name: Helix, dtype: object


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32761 entries, 0 to 32760
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Entry          32761 non-null  object
 1   Entry Name     32761 non-null  object
 2   Protein names  32761 non-null  object
 3   Gene Names     21071 non-null  object
 4   Organism       32761 non-null  object
 5   Length         32761 non-null  int64 
 6   Sequence       32761 non-null  object
 7   Reviewed       32761 non-null  object
 8   Helix          7642 non-null   object
 9   Turn           5059 non-null   object
 10  Beta strand    6887 non-null   object
 11  PubMed ID      31352 non-null  object
 12  DOI ID         31222 non-null  object
dtypes: int64(1), object(12)
memory usage: 3.2+ MB


In [ ]:
unique_values = df.nunique()
print("Unique values in each column:")
print(unique_values)

Unique values in each column:
Entry            32761
Entry Name       32761
Protein names    24023
Gene Names       18007
Organism          4413
Length             199
Sequence         30518
Reviewed             1
Helix             7638
Turn              5055
Beta strand       6884
PubMed ID        21368
DOI ID           21207
dtype: int64


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Embedding
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# import numpy as np

# # Preprocess the sequence data
# def encode_sequence(sequence, max_length, char_to_int):
#     encoded = [char_to_int[char] for char in sequence if char in char_to_int]
#     return pad_sequences([encoded], maxlen=max_length, padding='post')[0]

# # Prepare the data
# max_length = 200  # Maximum sequence length
# unique_chars = set(''.join(df['Sequence']))
# char_to_int = {char: idx + 1 for idx, char in enumerate(unique_chars)}  # Map chars to integers

# df['Encoded_Sequence'] = df['Sequence'].apply(lambda x: encode_sequence(x, max_length, char_to_int))

# # Prepare target variables
# target_columns = ['Helix', 'Turn', 'Beta strand']
# df[target_columns] = df[target_columns].fillna('')  # Replace NaN with empty strings

# # Encode target variables
# label_encoders = {col: LabelEncoder() for col in target_columns}
# for col in target_columns:
#     df[col] = label_encoders[col].fit_transform(df[col])

# # Split the data
# X = np.array(df['Encoded_Sequence'].tolist())
# y = df[target_columns].values
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Build the CNN model
# model = Sequential([
#     Embedding(input_dim=len(char_to_int) + 1, output_dim=128, input_length=max_length),
#     Conv1D(filters=64, kernel_size=3, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Conv1D(filters=128, kernel_size=3, activation='relu'),
#     MaxPooling1D(pool_size=2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(target_columns), activation='sigmoid')  # Multi-label classification
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=10,
#     batch_size=32
# )

# # Evaluate the model
# loss, accuracy = model.evaluate(X_val, y_val)
# print(f"Validation Loss: {loss}")
# print(f"Validation Accuracy: {accuracy}")

In [ ]:
df = df[['Sequence','Length', 'Organism', 'Helix', 'Turn', 'Beta strand']]
df

,Sequence,Length,Organism,Helix,Turn,Beta strand
0,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37,Conus betulinus (Beech cone),"HELIX 28..31; /evidence=""ECO:0007829|PDB:2NAY""",NaN,"STRAND 23..26; /evidence=""ECO:0007829|PDB:2NAY"""
1,MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...,134,Physalaemus nattereri (Cuyaba dwarf frog) (Eup...,NaN,NaN,NaN
2,MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...,184,Coprinopsis cinerea (Inky cap fungus) (Hormogr...,NaN,NaN,NaN
3,MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...,160,Cucumis sativus (Cucumber),NaN,NaN,NaN
4,MEIKYLLTVFLVLLIVSDHCQAFLFSLIPHAISGLISAFKGRRKRD...,74,Androctonus crassicauda (Arabian fat-tailed sc...,NaN,NaN,NaN
...,...,...,...,...,...,...
32756,MLEKVYVALIHYPIKGKDGSIISTAVTNLDVHDIARTARTYNLKGY...,192,Thermotoga maritima (strain ATCC 43589 / DSM 3...,"HELIX 28..40; /evidence=""ECO:0007829|PDB:3DCM""...","TURN 161..164; /evidence=""ECO:0007829|PDB:3DCM...","STRAND 4..10; /evidence=""ECO:0007829|PDB:3DCM""..."
32757,MTTSRRPSPRIRSFVKDLSATIPGAFRFTRGHYSMEELAREAIIRG...,180,Aeropyrum pernix (strain ATCC 700893 / DSM 118...,"HELIX 9..19; /evidence=""ECO:0007829|PDB:2CXH"";...",NaN,"STRAND 1..6; /evidence=""ECO:0007829|PDB:2CXH"";..."
32758,MAAKTKTLELEDNVFLLLEGNLKRIFATPIGYTTFREFQNVVFNCA...,184,Chlamydia pneumoniae (Chlamydophila pneumoniae),"HELIX 12..25; /evidence=""ECO:0007829|PDB:3Q9D""...",NaN,"STRAND 100..111; /evidence=""ECO:0007829|PDB:3Q..."
32759,MAISAEELEKILKKSFPSSVIKITDLVGDQDHYALEISDAQFNGLS...,77,Rickettsia prowazekii (strain Madrid E),"HELIX 5..12; /evidence=""ECO:0007829|PDB:2MCQ"";...","TURN 13..15; /evidence=""ECO:0007829|PDB:2MCQ"";...","STRAND 17..19; /evidence=""ECO:0007829|PDB:2MCQ..."


In [ ]:
# Fungsi untuk mengekstrak angka dari string dalam format "X..Y"
def extract_ranges(text):
    """Ekstrak semua range angka dalam format "X..Y" dari teks"""
    if pd.isna(text):
        return []
    pattern = r'(\d+)\.\.(\d+)'
    matches = re.findall(pattern, text)
    # Konversi setiap pasangan angka menjadi integer
    ranges = [(int(start), int(end)) for start, end in matches]
    return ranges

# Fungsi untuk menghitung jumlah struktur dalam setiap kolom
def count_structures(df, column_name):
    """Menghitung jumlah struktur dalam setiap baris untuk kolom tertentu"""
    if column_name not in df.columns:
        return 0

    counts = []
    for idx, row in df.iterrows():
        if pd.isna(row[column_name]):
            counts.append(0)
        else:
            structures = extract_ranges(row[column_name])
            counts.append(len(structures))

    return counts

# Hitung jumlah struktur untuk setiap baris
df['helix_count'] = count_structures(df, 'Helix')
df['turn_count'] = count_structures(df, 'Turn')
df['strand_count'] = count_structures(df, 'Beta strand')

# Hitung nilai maksimum dari jumlah struktur
max_helix = df['helix_count'].max() if 'helix_count' in df.columns else 0
max_turn = df['turn_count'].max() if 'turn_count' in df.columns else 0
max_strand = df['strand_count'].max() if 'strand_count' in df.columns else 0

print("Jumlah maksimum dari struktur:")
print(f"Maksimum Helix: {max_helix}")
print(f"Maksimum Turn: {max_turn}")
print(f"Maksimum Strand: {max_strand}")

# Fungsi untuk membuat kolom individual untuk setiap pasangan start-end
def create_structure_columns(df, column_name, prefix):
    """
    Membuat kolom individual untuk setiap struktur, misalnya:
    HELIX_1_start, HELIX_1_end, HELIX_2_start, HELIX_2_end, dst.
    """
    # Temukan jumlah maksimum struktur di semua baris
    max_structures = 0
    for idx, row in df.iterrows():
        if pd.isna(row[column_name]):
            continue
        structures = extract_ranges(row[column_name])
        max_structures = max(max_structures, len(structures))

    print(f"Jumlah maksimum struktur {prefix}: {max_structures}")

    # Buat kolom kosong dengan nilai NaN untuk semua struktur
    for i in range(1, max_structures + 1):
        df[f'{prefix}_{i}_start'] = np.nan
        df[f'{prefix}_{i}_end'] = np.nan

    # Isi kolom dengan nilai dari setiap baris
    for idx, row in df.iterrows():
        if pd.isna(row[column_name]):
            continue
        structures = extract_ranges(row[column_name])
        for i, (start, end) in enumerate(structures, 1):
            if i <= max_structures:  # Pastikan tidak melebihi jumlah kolom yang dibuat
                df.at[idx, f'{prefix}_{i}_start'] = start
                df.at[idx, f'{prefix}_{i}_end'] = end

    return df, max_structures

# Terapkan fungsi ke setiap kolom struktur dan dapatkan jumlah maksimum struktur
df, max_helix_cols = create_structure_columns(df, 'Helix', 'HELIX')
df, max_turn_cols = create_structure_columns(df, 'Turn', 'TURN')
df, max_strand_cols = create_structure_columns(df, 'Beta strand', 'STRAND')

# Tampilkan hasil dengan kolom baru - Helix
helix_cols = [col for col in df.columns if 'HELIX_' in col]
turn_cols = [col for col in df.columns if 'TURN_' in col]
strand_cols = [col for col in df.columns if 'STRAND_' in col]

# Hitung jumlah struktur untuk setiap baris
print("\nJumlah struktur per baris:")
print(df[['Sequence', 'helix_count', 'turn_count', 'strand_count']])

# Menampilkan statistik jumlah struktur
print("\nStatistik jumlah struktur:")
print("Helix:")
print(f"  Minimum: {df['helix_count'].min()}")
print(f"  Maksimum: {df['helix_count'].max()}")
print(f"  Rata-rata: {df['helix_count'].mean():.2f}")
print("Turn:")
print(f"  Minimum: {df['turn_count'].min()}")
print(f"  Maksimum: {df['turn_count'].max()}")
print(f"  Rata-rata: {df['turn_count'].mean():.2f}")
print("Strand:")
print(f"  Minimum: {df['strand_count'].min()}")
print(f"  Maksimum: {df['strand_count'].max()}")
print(f"  Rata-rata: {df['strand_count'].mean():.2f}")

# Contoh cara menggunakan data dari kolom individual
print("\nNilai dari kolom struktur untuk baris pertama:")
print(f"HELIX_1_start: {df['HELIX_1_start'][0]}, HELIX_1_end: {df['HELIX_1_end'][0]}")
if max_turn_cols >= 1:
    print(f"TURN_1_start: {df['TURN_1_start'][0]}, TURN_1_end: {df['TURN_1_end'][0]}")
if max_strand_cols >= 1:
    print(f"STRAND_1_start: {df['STRAND_1_start'][0]}, STRAND_1_end: {df['STRAND_1_end'][0]}")

# Menampilkan semua kolom untuk melihat struktur lengkap dataframe
print("\nSemua kolom dalam dataframe:")
print(df.columns.tolist())

# Cara menggunakan informasi jumlah maksimum struktur
print("\nJumlah maksimum struktur (dari fungsi):")
print(f"Maksimum kolom Helix: {max_helix_cols}")
print(f"Maksimum kolom Turn: {max_turn_cols}")
print(f"Maksimum kolom Strand: {max_strand_cols}")

Jumlah maksimum dari struktur:
Maksimum Helix: 17
Maksimum Turn: 7
Maksimum Strand: 22
Jumlah maksimum struktur HELIX: 17
Jumlah maksimum struktur TURN: 7
Jumlah maksimum struktur STRAND: 22

Jumlah struktur per baris:
                                                Sequence  helix_count  \
0                  PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG            1   
1      MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...            0   
2      MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...            0   
3      MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...            0   
4      MEIKYLLTVFLVLLIVSDHCQAFLFSLIPHAISGLISAFKGRRKRD...            0   
...                                                  ...          ...   
32756  MLEKVYVALIHYPIKGKDGSIISTAVTNLDVHDIARTARTYNLKGY...            8   
32757  MTTSRRPSPRIRSFVKDLSATIPGAFRFTRGHYSMEELAREAIIRG...            4   
32758  MAAKTKTLELEDNVFLLLEGNLKRIFATPIGYTTFREFQNVVFNCA...            7   
32759  MAISAEELEKILKKSFPSSVIKITDLVGDQDHYALEISDAQFNG

In [ ]:
import pandas as pd
import numpy as np

# Anggap df adalah dataset Anda
# df = pd.read_csv('your_dataset.csv')

# Fungsi umum untuk mengekstrak sequence berdasarkan indeks start dan end
def extract_sequence(row, start_col, end_col):
    # Periksa apakah start dan end bukan NaN
    if pd.notna(row[start_col]) and pd.notna(row[end_col]):
        # Convert ke integer dan kurangi 1 jika indeks dimulai dari 0
        # (tergantung pada apakah indeks dalam dataset dimulai dari 0 atau 1)
        start_idx = int(row[start_col]) - 1  # kurangi 1 jika indeks dimulai dari 0
        end_idx = int(row[end_col])  # tidak dikurangi jika end_idx inklusif

        # Ekstrak sequence berdasarkan indeks
        if pd.notna(row['Sequence']):
            return row['Sequence'][start_idx:end_idx]
    return np.nan

# 1. Membuat kolom HELIX_n
helix_cols = [col for col in df.columns if col.startswith('HELIX_') and '_start' in col]
max_helix = len(helix_cols)

for i in range(1, max_helix + 1):
    start_col = f'HELIX_{i}_start'
    end_col = f'HELIX_{i}_end'
    new_col = f'HELIX_{i}'

    if start_col in df.columns and end_col in df.columns:
        df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)

# 2. Membuat kolom TURN_n
turn_cols = [col for col in df.columns if col.startswith('TURN_') and '_start' in col]
max_turn = len(turn_cols)

for i in range(1, max_turn + 1):
    start_col = f'TURN_{i}_start'
    end_col = f'TURN_{i}_end'
    new_col = f'TURN_{i}'

    if start_col in df.columns and end_col in df.columns:
        df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)

# 3. Membuat kolom STRAND_n
strand_cols = [col for col in df.columns if col.startswith('STRAND_') and '_start' in col]
max_strand = len(strand_cols)

for i in range(1, max_strand + 1):
    start_col = f'STRAND_{i}_start'
    end_col = f'STRAND_{i}_end'
    new_col = f'STRAND_{i}'

    if start_col in df.columns and end_col in df.columns:
        df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)

# Menampilkan contoh hasilnya
print(df.head())

<ipython-input-167-023035dc7b6f>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)
<ipython-input-167-023035dc7b6f>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)
<ipython-input-167-023035dc7b6f>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

                                            Sequence  Length  \
0              PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG      37   
1  MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...     134   
2  MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...     184   
3  MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...     160   
4  MEIKYLLTVFLVLLIVSDHCQAFLFSLIPHAISGLISAFKGRRKRD...      74   

                                            Organism  \
0                       Conus betulinus (Beech cone)   
1  Physalaemus nattereri (Cuyaba dwarf frog) (Eup...   
2  Coprinopsis cinerea (Inky cap fungus) (Hormogr...   
3                         Cucumis sativus (Cucumber)   
4  Androctonus crassicauda (Arabian fat-tailed sc...   

                                            Helix Turn  \
0  HELIX 28..31; /evidence="ECO:0007829|PDB:2NAY"  NaN   
1                                             NaN  NaN   
2                                             NaN  NaN   
3                                             

In [ ]:
import pandas as pd

# Mengatur agar semua kolom ditampilkan
pd.set_option('display.max_columns', None)

# Menampilkan 5 baris pertama dengan semua kolom
display(df.head())


,Sequence,Length,Organism,Helix,Turn,Beta strand,helix_count,turn_count,strand_count,HELIX_1_start,HELIX_1_end,HELIX_2_start,HELIX_2_end,HELIX_3_start,HELIX_3_end,HELIX_4_start,HELIX_4_end,HELIX_5_start,HELIX_5_end,HELIX_6_start,HELIX_6_end,HELIX_7_start,HELIX_7_end,HELIX_8_start,HELIX_8_end,HELIX_9_start,HELIX_9_end,HELIX_10_start,HELIX_10_end,HELIX_11_start,HELIX_11_end,HELIX_12_start,HELIX_12_end,HELIX_13_start,HELIX_13_end,HELIX_14_start,HELIX_14_end,HELIX_15_start,HELIX_15_end,HELIX_16_start,HELIX_16_end,HELIX_17_start,HELIX_17_end,TURN_1_start,TURN_1_end,TURN_2_start,TURN_2_end,TURN_3_start,TURN_3_end,TURN_4_start,TURN_4_end,TURN_5_start,TURN_5_end,TURN_6_start,TURN_6_end,TURN_7_start,TURN_7_end,STRAND_1_start,STRAND_1_end,STRAND_2_start,STRAND_2_end,STRAND_3_start,STRAND_3_end,STRAND_4_start,STRAND_4_end,STRAND_5_start,STRAND_5_end,STRAND_6_start,STRAND_6_end,STRAND_7_start,STRAND_7_end,STRAND_8_start,STRAND_8_end,STRAND_9_start,STRAND_9_end,STRAND_10_start,STRAND_10_end,STRAND_11_start,STRAND_11_end,STRAND_12_start,STRAND_12_end,STRAND_13_start,STRAND_13_end,STRAND_14_start,STRAND_14_end,STRAND_15_start,STRAND_15_end,STRAND_16_start,STRAND_16_end,STRAND_17_start,STRAND_17_end,STRAND_18_start,STRAND_18_end,STRAND_19_start,STRAND_19_end,STRAND_20_start,STRAND_20_end,STRAND_21_start,STRAND_21_end,STRAND_22_start,STRAND_22_end,HELIX_1,HELIX_2,HELIX_3,HELIX_4,HELIX_5,HELIX_6,HELIX_7,HELIX_8,HELIX_9,HELIX_10,HELIX_11,HELIX_12,HELIX_13,HELIX_14,HELIX_15,HELIX_16,HELIX_17,TURN_1,TURN_2,TURN_3,TURN_4,TURN_5,TURN_6,TURN_7,STRAND_1,STRAND_2,STRAND_3,STRAND_4,STRAND_5,STRAND_6,STRAND_7,STRAND_8,STRAND_9,STRAND_10,STRAND_11,STRAND_12,STRAND_13,STRAND_14,STRAND_15,STRAND_16,STRAND_17,STRAND_18,STRAND_19,STRAND_20,STRAND_21,STRAND_22
0,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37,Conus betulinus (Beech cone),"HELIX 28..31; /evidence=""ECO:0007829|PDB:2NAY""",NaN,"STRAND 23..26; /evidence=""ECO:0007829|PDB:2NAY""",1,0,1,28.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CILN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CSNP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...,134,Physalaemus nattereri (Cuyaba dwarf frog) (Eup...,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...,184,Coprinopsis cinerea (Inky cap fungus) (Hormogr...,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...,160,Cucumis sativus (Cucumber),NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
# prompt: tampilkan df dengan length = 5 AA dan helix atau strand atau turn tidak null

# Filter the DataFrame
filtered_df = df[(df['Length'] == 7) &
                 ((df['Helix'].notnull()) | (df['Turn'].notnull()) | (df['Beta strand'].notnull()))]

# Display the filtered DataFrame
display(filtered_df)


,Sequence,Length,Organism,Helix,Turn,Beta strand,helix_count,turn_count,strand_count,HELIX_1_start,HELIX_1_end,HELIX_2_start,HELIX_2_end,HELIX_3_start,HELIX_3_end,HELIX_4_start,HELIX_4_end,HELIX_5_start,HELIX_5_end,HELIX_6_start,HELIX_6_end,HELIX_7_start,HELIX_7_end,HELIX_8_start,HELIX_8_end,HELIX_9_start,HELIX_9_end,HELIX_10_start,HELIX_10_end,HELIX_11_start,HELIX_11_end,HELIX_12_start,HELIX_12_end,HELIX_13_start,HELIX_13_end,HELIX_14_start,HELIX_14_end,HELIX_15_start,HELIX_15_end,HELIX_16_start,HELIX_16_end,HELIX_17_start,HELIX_17_end,TURN_1_start,TURN_1_end,TURN_2_start,TURN_2_end,TURN_3_start,TURN_3_end,TURN_4_start,TURN_4_end,TURN_5_start,TURN_5_end,TURN_6_start,TURN_6_end,TURN_7_start,TURN_7_end,STRAND_1_start,STRAND_1_end,STRAND_2_start,STRAND_2_end,STRAND_3_start,STRAND_3_end,STRAND_4_start,STRAND_4_end,STRAND_5_start,STRAND_5_end,STRAND_6_start,STRAND_6_end,STRAND_7_start,STRAND_7_end,STRAND_8_start,STRAND_8_end,STRAND_9_start,STRAND_9_end,STRAND_10_start,STRAND_10_end,STRAND_11_start,STRAND_11_end,STRAND_12_start,STRAND_12_end,STRAND_13_start,STRAND_13_end,STRAND_14_start,STRAND_14_end,STRAND_15_start,STRAND_15_end,STRAND_16_start,STRAND_16_end,STRAND_17_start,STRAND_17_end,STRAND_18_start,STRAND_18_end,STRAND_19_start,STRAND_19_end,STRAND_20_start,STRAND_20_end,STRAND_21_start,STRAND_21_end,STRAND_22_start,STRAND_22_end,HELIX_1,HELIX_2,HELIX_3,HELIX_4,HELIX_5,HELIX_6,HELIX_7,HELIX_8,HELIX_9,HELIX_10,HELIX_11,HELIX_12,HELIX_13,HELIX_14,HELIX_15,HELIX_16,HELIX_17,TURN_1,TURN_2,TURN_3,TURN_4,TURN_5,TURN_6,TURN_7,STRAND_1,STRAND_2,STRAND_3,STRAND_4,STRAND_5,STRAND_6,STRAND_7,STRAND_8,STRAND_9,STRAND_10,STRAND_11,STRAND_12,STRAND_13,STRAND_14,STRAND_15,STRAND_16,STRAND_17,STRAND_18,STRAND_19,STRAND_20,STRAND_21,STRAND_22
20956,MRTGNAN,7,Escherichia coli,NaN,NaN,"STRAND 3..5; /evidence=""ECO:0007829|PDB:3H9G""",0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TGN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31629,LVTLVFV,7,Enterococcus faecalis (Streptococcus faecalis),NaN,NaN,"STRAND 2..4; /evidence=""ECO:0007829|PDB:4FAJ""",0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VTL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# prompt: hapus kolom  'helix_count', 'turn_count', 'strand_count', 'HELIX_1_start', 'HELIX_1_end', 'HELIX_2_start', 'HELIX_2_end', 'HELIX_3_start', 'HELIX_3_end', 'HELIX_4_start', 'HELIX_4_end', 'HELIX_5_start', 'HELIX_5_end', 'HELIX_6_start', 'HELIX_6_end', 'HELIX_7_start', 'HELIX_7_end', 'HELIX_8_start', 'HELIX_8_end', 'HELIX_9_start', 'HELIX_9_end', 'HELIX_10_start', 'HELIX_10_end', 'HELIX_11_start', 'HELIX_11_end', 'HELIX_12_start', 'HELIX_12_end', 'HELIX_13_start', 'HELIX_13_end', 'HELIX_14_start', 'HELIX_14_end', 'HELIX_15_start', 'HELIX_15_end', 'HELIX_16_start', 'HELIX_16_end', 'HELIX_17_start', 'HELIX_17_end', 'TURN_1_start', 'TURN_1_end', 'TURN_2_start', 'TURN_2_end', 'TURN_3_start', 'TURN_3_end', 'TURN_4_start', 'TURN_4_end', 'TURN_5_start', 'TURN_5_end', 'TURN_6_start', 'TURN_6_end', 'TURN_7_start', 'TURN_7_end', 'STRAND_1_start', 'STRAND_1_end', 'STRAND_2_start', 'STRAND_2_end', 'STRAND_3_start', 'STRAND_3_end', 'STRAND_4_start', 'STRAND_4_end', 'STRAND_5_start', 'STRAND_5_end', 'STRAND_6_start', 'STRAND_6_end', 'STRAND_7_start', 'STRAND_7_end', 'STRAND_8_start', 'STRAND_8_end', 'STRAND_9_start', 'STRAND_9_end', 'STRAND_10_start', 'STRAND_10_end', 'STRAND_11_start', 'STRAND_11_end', 'STRAND_12_start', 'STRAND_12_end', 'STRAND_13_start', 'STRAND_13_end', 'STRAND_14_start', 'STRAND_14_end', 'STRAND_15_start', 'STRAND_15_end', 'STRAND_16_start', 'STRAND_16_end', 'STRAND_17_start', 'STRAND_17_end', 'STRAND_18_start', 'STRAND_18_end', 'STRAND_19_start', 'STRAND_19_end', 'STRAND_20_start', 'STRAND_20_end', 'STRAND_21_start', 'STRAND_21_end', 'STRAND_22_start', 'STRAND_22_end'

cols_to_drop = ['Helix', 'Turn', 'Beta strand', 'HELIX_1_start', 'HELIX_1_end', 'HELIX_2_start', 'HELIX_2_end', 'HELIX_3_start', 'HELIX_3_end', 'HELIX_4_start', 'HELIX_4_end', 'HELIX_5_start', 'HELIX_5_end', 'HELIX_6_start', 'HELIX_6_end', 'HELIX_7_start', 'HELIX_7_end', 'HELIX_8_start', 'HELIX_8_end', 'HELIX_9_start', 'HELIX_9_end', 'HELIX_10_start', 'HELIX_10_end', 'HELIX_11_start', 'HELIX_11_end', 'HELIX_12_start', 'HELIX_12_end', 'HELIX_13_start', 'HELIX_13_end', 'HELIX_14_start', 'HELIX_14_end', 'HELIX_15_start', 'HELIX_15_end', 'HELIX_16_start', 'HELIX_16_end', 'HELIX_17_start', 'HELIX_17_end', 'TURN_1_start', 'TURN_1_end', 'TURN_2_start', 'TURN_2_end', 'TURN_3_start', 'TURN_3_end', 'TURN_4_start', 'TURN_4_end', 'TURN_5_start', 'TURN_5_end', 'TURN_6_start', 'TURN_6_end', 'TURN_7_start', 'TURN_7_end', 'STRAND_1_start', 'STRAND_1_end', 'STRAND_2_start', 'STRAND_2_end', 'STRAND_3_start', 'STRAND_3_end', 'STRAND_4_start', 'STRAND_4_end', 'STRAND_5_start', 'STRAND_5_end', 'STRAND_6_start', 'STRAND_6_end', 'STRAND_7_start', 'STRAND_7_end', 'STRAND_8_start', 'STRAND_8_end', 'STRAND_9_start', 'STRAND_9_end', 'STRAND_10_start', 'STRAND_10_end', 'STRAND_11_start', 'STRAND_11_end', 'STRAND_12_start', 'STRAND_12_end', 'STRAND_13_start', 'STRAND_13_end', 'STRAND_14_start', 'STRAND_14_end', 'STRAND_15_start', 'STRAND_15_end', 'STRAND_16_start', 'STRAND_16_end', 'STRAND_17_start', 'STRAND_17_end', 'STRAND_18_start', 'STRAND_18_end', 'STRAND_19_start', 'STRAND_19_end', 'STRAND_20_start', 'STRAND_20_end', 'STRAND_21_start', 'STRAND_21_end', 'STRAND_22_start', 'STRAND_22_end']
df = df.drop(columns=cols_to_drop, errors='ignore')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32761 entries, 0 to 32760
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Sequence      32761 non-null  object
 1   Length        32761 non-null  int64 
 2   Organism      32761 non-null  object
 3   helix_count   32761 non-null  int64 
 4   turn_count    32761 non-null  int64 
 5   strand_count  32761 non-null  int64 
 6   HELIX_1       7642 non-null   object
 7   HELIX_2       6547 non-null   object
 8   HELIX_3       5495 non-null   object
 9   HELIX_4       4389 non-null   object
 10  HELIX_5       3385 non-null   object
 11  HELIX_6       2494 non-null   object
 12  HELIX_7       1740 non-null   object
 13  HELIX_8       1152 non-null   object
 14  HELIX_9       745 non-null    object
 15  HELIX_10      427 non-null    object
 16  HELIX_11      210 non-null    object
 17  HELIX_12      98 non-null     object
 18  HELIX_13      42 non-null     object
 19  HELI

In [ ]:
import pandas as pd
import numpy as np

# Misalkan df adalah dataframe asli Anda
# df = pd.read_csv('your_data.csv')  # Ganti dengan kode untuk membaca data Anda

# Membuat dataframe kosong untuk hasil akhir
result_df = pd.DataFrame(columns=['length_seq', 'Organism', 'subsequence', 'length_sub_seq', 'type'])

# Ambil kolom utama yang akan disertakan di semua baris
lengths = df['Length'].values
organisms = df['Organism'].values
sequences = df['Sequence'].values  # Jika diperlukan

# Fungsi untuk memproses kolom dan menambahkan ke dataframe hasil
def process_columns(df, prefix, type_name):
    rows_list = []

    # Ambil jumlah entri dalam dataframe asli
    n_entries = len(df)

    # Cari kolom yang cocok dengan prefix (HELIX_, TURN_, atau STRAND_)
    cols = [col for col in df.columns if col.startswith(prefix)]

    # Untuk setiap entri dalam dataframe asli
    for i in range(n_entries):
        length_seq = df.loc[i, 'Length']
        organism = df.loc[i, 'Organism']

        # Untuk setiap kolom yang cocok dengan prefix
        for col in cols:
            # Ambil nilai sub-sequence, lewati jika NaN
            sub_seq = df.loc[i, col]
            if pd.notna(sub_seq):
                # Hitung panjang sub-sequence
                length_sub_seq = len(str(sub_seq))

                rows_list.append({
                    'length_seq': length_seq,
                    'Organism': organism,
                    'subsequence': sub_seq,
                    'length_sub_seq': length_sub_seq,
                    'type': type_name
                })

    return pd.DataFrame(rows_list)

# Proses setiap tipe struktur
helix_df = process_columns(df, 'HELIX_', 'helix')
turn_df = process_columns(df, 'TURN_', 'turn')
strand_df = process_columns(df, 'STRAND_', 'strand')

# Gabungkan semua hasil
result_df = pd.concat([helix_df, turn_df, strand_df], ignore_index=True)

# Tampilkan informasi tentang dataframe hasil
print(result_df.info())
print("\nContoh beberapa baris pertama:")
print(result_df.head())

# Jika ingin menyimpan hasil ke file
# result_df.to_csv('transformed_data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81989 entries, 0 to 81988
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   length_seq      81989 non-null  int64 
 1   Organism        81989 non-null  object
 2   subsequence     81989 non-null  object
 3   length_sub_seq  81989 non-null  int64 
 4   type            81989 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.1+ MB
None

Contoh beberapa baris pertama:
   length_seq                                       Organism  \
0          37                   Conus betulinus (Beech cone)   
1         147  Aphanizomenon flos-aquae (strain 2012/KM1/D3)   
2         147  Aphanizomenon flos-aquae (strain 2012/KM1/D3)   
3         147  Aphanizomenon flos-aquae (strain 2012/KM1/D3)   
4         147  Aphanizomenon flos-aquae (strain 2012/KM1/D3)   

                     subsequence  length_sub_seq   type  
0                           CILN               4  heli

In [ ]:
result_df

In [ ]:
# prompt: rata2 length_seq dan length_sub_seq

# Calculate statistics for 'length_seq' and 'length_sub_seq'
print("\nStatistics for length_seq and length_sub_seq:")
print("length_seq:")
print(f"  Minimum: {result_df['length_seq'].min()}")
print(f"  Maximum: {result_df['length_seq'].max()}")
print(f"  Average: {result_df['length_seq'].mean():.2f}")
print("length_sub_seq:")
print(f"  Minimum: {result_df['length_sub_seq'].min()}")
print(f"  Maximum: {result_df['length_sub_seq'].max()}")
print(f"  Average: {result_df['length_sub_seq'].mean():.2f}")



Statistics for length_seq and length_sub_seq:
length_seq:
  Minimum: 7
  Maximum: 200
  Average: 143.04
length_sub_seq:
  Minimum: 3
  Maximum: 126
  Average: 7.33


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load the DataFrame (assuming df is already loaded)

def perform_anova(df):
    # Check if the required columns are in the DataFrame
    if 'length_seq' not in df.columns or 'type' not in df.columns:
        print("DataFrame must contain 'length_seq' and 'type' columns.")
        return

    try:
        # Convert the 'type' column to a categorical type if it's not
        df['type'] = df['type'].astype('category')

        # ANOVA using OLS
        model = ols('length_seq ~ C(type)', data=df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)

        print("ANOVA Results:")
        print(anova_table)
        return anova_table

    except Exception as e:
        print("Error while performing ANOVA:", str(e))

# Example usage
anova_result = perform_anova(result_df)


ANOVA Results:
                sum_sq       df          F        PR(>F)
C(type)   1.995775e+05      2.0  67.115624  7.514326e-30
Residual  1.218983e+08  81986.0        NaN           NaN


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load the DataFrame (assuming df is already loaded)

def perform_anova(df):
    # Check if the required columns are in the DataFrame
    if 'length_sub_seq' not in df.columns or 'type' not in df.columns:
        print("DataFrame must contain 'length_sub_seq' and 'type' columns.")
        return

    try:
        # Convert the 'type' column to a categorical type if it's not
        df['type'] = df['type'].astype('category')

        # ANOVA using OLS
        model = ols('length_sub_seq ~ C(type)', data=df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)

        print("ANOVA Results:")
        print(anova_table)
        return anova_table

    except Exception as e:
        print("Error while performing ANOVA:", str(e))

# Example usage
anova_result = perform_anova(result_df)


ANOVA Results:
                sum_sq       df            F  PR(>F)
C(type)   4.140723e+05      2.0  8218.524463     0.0
Residual  2.065342e+06  81986.0          NaN     NaN


In [ ]:
# prompt: drop length_seq di result_df

# Drop the specified columns
cols_to_drop = ['length_seq']
result_df = result_df.drop(columns=cols_to_drop, errors='ignore')


In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Generate a contingency table between organism and type
contingency_table = pd.crosstab(result_df['Organism'], result_df['type'])

# Perform the Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Display results
print(f"Chi-Square Statistic: {chi2}")
print(f"P-Value: {p}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

# Interpret the result
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant association between organism and type.")
else:
    print("Fail to reject the null hypothesis: No significant association between organism and type.")


Chi-Square Statistic: 7264.679715675109
P-Value: 0.0
Degrees of Freedom: 2858
Expected Frequencies:
type                                                    helix     strand  \
Organism                                                                   
Acanthamoeba castellanii (Amoeba)                   20.551684  23.079108   
Acanthamoeba polyphaga mimivirus (APMV)              5.452488   6.123029   
Acanthoscurria gomesiana (Tarantula spider) (Ph...   0.838844   0.942004   
Acaryochloris marina (strain MBIC 11017)             3.774799   4.239020   
Acetivibrio thermocellus (strain ATCC 27405 / D...  18.873995  21.195099   
...                                                       ...        ...   
Zea mays subsp. parviglumis (Balsas teosinte)        3.355377   3.768018   
Zoarces americanus (Ocean pout) (Macrozoarces a...   6.291332   7.065033   
Zymomonas mobilis subsp. mobilis (strain ATCC 1...   5.871910   6.594031   
Zymomonas mobilis subsp. mobilis (strain ATCC 3...   4.194221   

In [ ]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81989 entries, 0 to 81988
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Organism        81989 non-null  object  
 1   subsequence     81989 non-null  object  
 2   length_sub_seq  81989 non-null  int64   
 3   type            81989 non-null  category
dtypes: category(1), int64(1), object(2)
memory usage: 2.0+ MB


In [ ]:
result_df.head(5)

,Organism,subsequence,length_sub_seq,type
0,Conus betulinus (Beech cone),CILN,4,helix
1,Aphanizomenon flos-aquae (strain 2012/KM1/D3),EPVIIETRLELIGRYLDHL,19,helix
2,Aphanizomenon flos-aquae (strain 2012/KM1/D3),KKF,3,helix
3,Aphanizomenon flos-aquae (strain 2012/KM1/D3),LDDYL,5,helix
4,Aphanizomenon flos-aquae (strain 2012/KM1/D3),FEQQLITERLLQLITQAAIDINDHILSKL,29,helix


In [ ]:
result_df.describe()

,length_sub_seq
count,81989.000000
mean,7.325629
std,5.499199
min,3.000000
25%,3.000000
50%,6.000000
75%,9.000000
max,126.000000


In [ ]:
# prompt: jumlah unique value dari setiap kolom di result_df

print(result_df.nunique())


Organism           1430
subsequence       59245
length_sub_seq       76
type                  3
dtype: int64


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [ ]:
# # prompt: hitung outliers dengan iqr dan buang outlier

# # Calculate IQR for 'Length' column in the training set
# Q1 = train_set['Length'].quantile(0.25)
# Q3 = train_set['Length'].quantile(0.75)
# IQR = Q3 - Q1

# # Define bounds for outliers
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# # Remove outliers from the training set
# train_set_no_outliers = train_set[~((train_set['Length'] < lower_bound) | (train_set['Length'] > upper_bound))]

# # Calculate IQR for 'Length' column in the validation set
# Q1_val = val_set['Length'].quantile(0.25)
# Q3_val = val_set['Length'].quantile(0.75)
# IQR_val = Q3_val - Q1_val

# # Define bounds for outliers in the validation set
# lower_bound_val = Q1_val - 1.5 * IQR_val
# upper_bound_val = Q3_val + 1.5 * IQR_val

# # Remove outliers from the validation set
# val_set_no_outliers = val_set[~((val_set['Length'] < lower_bound_val) | (val_set['Length'] > upper_bound_val))]

# # Print the shapes of the datasets before and after outlier removal
# print("Original training set shape:", train_set.shape)
# print("Training set shape after outlier removal:", train_set_no_outliers.shape)
# print("Original validation set shape:", val_set.shape)
# print("Validation set shape after outlier removal:", val_set_no_outliers.shape)


Dipisah-pisah dengan tujuan si yang label 0 ga kehapus karena dianggap outlier

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
# Drop duplicate rows from result_df
result_df = result_df.drop_duplicates()


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

Drop Column that Useless

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on the numerical features of the training set
# Assuming 'length_sub_seq' is the only numerical feature
scaler.fit(result_df[['length_sub_seq']])

# Transform the numerical features of the validation set using the same scaler
result_df['length_sub_seq'] = scaler.transform(result_df[['length_sub_seq']])

<ipython-input-184-1168cf028447>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['length_sub_seq'] = scaler.transform(result_df[['length_sub_seq']])


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Membuat objek LabelEncoder
label_encoder = LabelEncoder()

# Melakukan label encoding pada kolom 'type'
result_df['type_Encoded'] = label_encoder.fit_transform(result_df['type'])

<ipython-input-185-fd8a8ee088a6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['type_Encoded'] = label_encoder.fit_transform(result_df['type'])


In [ ]:
result_df = result_df.drop(columns=['type'], errors='ignore')


In [ ]:
result_df

,Organism,subsequence,length_sub_seq,type_Encoded
0,Conus betulinus (Beech cone),CILN,-0.616144,0
1,Aphanizomenon flos-aquae (strain 2012/KM1/D3),EPVIIETRLELIGRYLDHL,2.094104,0
2,Aphanizomenon flos-aquae (strain 2012/KM1/D3),KKF,-0.796827,0
3,Aphanizomenon flos-aquae (strain 2012/KM1/D3),LDDYL,-0.435461,0
4,Aphanizomenon flos-aquae (strain 2012/KM1/D3),FEQQLITERLLQLITQAAIDINDHILSKL,3.900936,0
...,...,...,...,...
81984,Chlamydia pneumoniae (Chlamydophila pneumoniae),EFLLMTD,-0.074094,1
81985,Rickettsia prowazekii (strain Madrid E),PSS,-0.796827,1
81986,Rickettsia prowazekii (strain Madrid E),KITDLVGDQD,0.467955,1
81987,Rickettsia prowazekii (strain Madrid E),YALEISDAQFNG,0.829322,1


In [ ]:
# prompt: cetak berapa persentase Organism yang memiliki frekuensi kemunculan 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 hingga akhir

# Menghitung frekuensi kemunculan setiap Organism
organism_counts = result_df['Organism'].value_counts()

# Menghitung total jumlah Organism
total_organisms = organism_counts.sum()

# Menghitung persentase untuk setiap frekuensi
frequency_percentages = {}
for count in range(1, organism_counts.max() + 1):
    organisms_with_this_count = organism_counts[organism_counts == count]
    num_organisms = len(organisms_with_this_count)
    percentage = (num_organisms / len(organism_counts)) * 100 if len(organism_counts) > 0 else 0
    frequency_percentages[count] = percentage

# Mencetak persentase untuk setiap frekuensi kemunculan
print("Persentase Organism berdasarkan frekuensi kemunculan:")
for count, percentage in frequency_percentages.items():
    print(f"Frekuensi {count}: {percentage:.2f}%")

Persentase Organism berdasarkan frekuensi kemunculan:
Frekuensi 1: 4.13%
Frekuensi 2: 4.13%
Frekuensi 3: 3.43%
Frekuensi 4: 3.85%
Frekuensi 5: 4.20%
Frekuensi 6: 3.71%
Frekuensi 7: 2.87%
Frekuensi 8: 4.13%
Frekuensi 9: 3.29%
Frekuensi 10: 5.38%
Frekuensi 11: 4.06%
Frekuensi 12: 4.06%
Frekuensi 13: 4.20%
Frekuensi 14: 4.13%
Frekuensi 15: 2.87%
Frekuensi 16: 3.29%
Frekuensi 17: 2.45%
Frekuensi 18: 2.45%
Frekuensi 19: 1.61%
Frekuensi 20: 1.61%
Frekuensi 21: 1.54%
Frekuensi 22: 1.89%
Frekuensi 23: 1.12%
Frekuensi 24: 1.12%
Frekuensi 25: 0.84%
Frekuensi 26: 0.77%
Frekuensi 27: 0.70%
Frekuensi 28: 1.19%
Frekuensi 29: 0.70%
Frekuensi 30: 0.42%
Frekuensi 31: 0.77%
Frekuensi 32: 0.84%
Frekuensi 33: 0.63%
Frekuensi 34: 0.70%
Frekuensi 35: 0.42%
Frekuensi 36: 0.56%
Frekuensi 37: 0.28%
Frekuensi 38: 0.56%
Frekuensi 39: 0.56%
Frekuensi 40: 0.49%
Frekuensi 41: 0.63%
Frekuensi 42: 0.14%
Frekuensi 43: 0.07%
Frekuensi 44: 0.21%
Frekuensi 45: 0.35%
Frekuensi 46: 0.28%
Frekuensi 47: 0.42%
Frekuensi 48: 0

In [ ]:
# prompt: hitung total persentase Organism yang memiliki frekuensi 1-10

# Filter Organism yang memiliki frekuensi 1 hingga 10
organisms_freq_1_to_10 = organism_counts[(organism_counts >= 1) & (organism_counts <= 10)]

# Menghitung jumlah Organism yang memiliki frekuensi 1 hingga 10
num_organisms_freq_1_to_10 = len(organisms_freq_1_to_10)

# Menghitung total jumlah Organism unik
total_unique_organisms = len(organism_counts)

# Menghitung total persentase Organism yang memiliki frekuensi 1-10
total_percentage_1_to_10 = (num_organisms_freq_1_to_10 / total_unique_organisms) * 100 if total_unique_organisms > 0 else 0

print(f"\nJumlah Organism dengan frekuensi 1-10: {num_organisms_freq_1_to_10}")
print(f"Total Organism unik: {total_unique_organisms}")
print(f"Total persentase Organism dengan frekuensi 1-10: {total_percentage_1_to_10:.2f}%")



Jumlah Organism dengan frekuensi 1-10: 559
Total Organism unik: 1430
Total persentase Organism dengan frekuensi 1-10: 39.09%


In [ ]:
# prompt: ubah nilai Organism yang memiliki frekuensi < 11  menjadi 'Other'

# Ubah nilai 'Organism' yang memiliki frekuensi < 11 menjadi 'Other'
low_frequency_organisms = organism_counts[organism_counts < 11].index
result_df['Organism'] = result_df['Organism'].apply(lambda x: 'Other' if x in low_frequency_organisms else x)

# Periksa hasilnya dengan melihat frekuensi 'Organism' setelah perubahan
print("\nFrekuensi Organism setelah mengubah frekuensi < 11 menjadi 'Other':")
print(result_df['Organism'].value_counts())


Frekuensi Organism setelah mengubah frekuensi < 10 menjadi 'Other':
Organism
Homo sapiens (Human)                                                                        10258
Escherichia coli (strain K12)                                                                5547
Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast)                        3625
Mus musculus (Mouse)                                                                         3398
Bacillus subtilis (strain 168)                                                               3105
                                                                                            ...  
Hepatitis B virus genotype D subtype adw (isolate United Kingdom/adyw/1979) (HBV-D)            10
Hepatitis B virus genotype A2 subtype adw (isolate Japan/Nishioka/1983) (HBV-A)                10
Oryza sativa subsp. indica (Rice)                                                              10
Culex quinquefasciatus (Southern house m

In [ ]:
# prompt: cetak result_df yang memiliki Organism other

result_df[result_df['Organism'] == 'Other']


,Organism,subsequence,length_sub_seq,type_Encoded
0,Other,CILN,-0.616144,0
17,Other,EELDAFHQIFTTTKEAIERFMAMLTPVIE,3.900936,0
18,Other,DHERLYYHHIYEEEEQRLSRLDVLIPLIEKFQDE,4.804352,0
19,Other,NNAFNRLLQELNLEKFGLHNFIEHVDLALFS,4.262302,0
20,Other,EERQTLLKELRKDAYEGYQYVKEKLAE,3.539569,0
...,...,...,...,...
81833,Other,EFKIKFKDPLKVT,1.010005,1
81834,Other,DSSNSV,-0.254778,1
81835,Other,ITIDQYTYD,0.287272,1
81836,Other,DDD,-0.796827,1


In [ ]:
# prompt: cetak berapa persentase subsequence yang memiliki frekuensi kemunculan 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 hingga akhir

# Hitung frekuensi kemunculan setiap subsequence
subsequence_counts = result_df['subsequence'].value_counts()

# Hitung total jumlah subsequence
total_subsequences = subsequence_counts.sum()

# Hitung persentase untuk setiap frekuensi
frequency_percentages = {}
for count in range(1, subsequence_counts.max() + 1):
    subsequences_with_this_count = subsequence_counts[subsequence_counts == count]
    num_subsequences = len(subsequences_with_this_count)
    percentage = (num_subsequences / len(subsequence_counts)) * 100 if len(subsequence_counts) > 0 else 0
    frequency_percentages[count] = percentage

# Mencetak persentase untuk setiap frekuensi kemunculan
print("Persentase subsequence berdasarkan frekuensi kemunculan:")
for count, percentage in frequency_percentages.items():
    print(f"Frekuensi {count}: {percentage:.2f}%")

Persentase subsequence berdasarkan frekuensi kemunculan:
Frekuensi 1: 87.17%
Frekuensi 2: 6.51%
Frekuensi 3: 2.28%
Frekuensi 4: 1.25%
Frekuensi 5: 0.76%
Frekuensi 6: 0.50%
Frekuensi 7: 0.40%
Frekuensi 8: 0.27%
Frekuensi 9: 0.21%
Frekuensi 10: 0.16%
Frekuensi 11: 0.12%
Frekuensi 12: 0.08%
Frekuensi 13: 0.06%
Frekuensi 14: 0.06%
Frekuensi 15: 0.04%
Frekuensi 16: 0.02%
Frekuensi 17: 0.03%
Frekuensi 18: 0.02%
Frekuensi 19: 0.01%
Frekuensi 20: 0.00%
Frekuensi 21: 0.01%
Frekuensi 22: 0.00%
Frekuensi 23: 0.00%
Frekuensi 24: 0.00%
Frekuensi 25: 0.00%
Frekuensi 26: 0.00%
Frekuensi 27: 0.00%
Frekuensi 28: 0.00%
Frekuensi 29: 0.00%
Frekuensi 30: 0.00%
Frekuensi 31: 0.00%
Frekuensi 32: 0.00%
Frekuensi 33: 0.00%
Frekuensi 34: 0.00%
Frekuensi 35: 0.00%
Frekuensi 36: 0.00%
Frekuensi 37: 0.00%
Frekuensi 38: 0.00%
Frekuensi 39: 0.00%


In [ ]:
# prompt: ubah nilai subsequence yang memiliki frekuensi < 2  menjadi 'Other'

# Ubah nilai 'subsequence' yang memiliki frekuensi < 2 menjadi 'Other'
low_frequency_subsequences = subsequence_counts[subsequence_counts < 2].index
result_df['subsequence'] = result_df['subsequence'].apply(lambda x: 'Other' if x in low_frequency_subsequences else x)

# Periksa hasilnya dengan melihat frekuensi 'subsequence' setelah perubahan
print("\nFrekuensi subsequence setelah mengubah frekuensi < 2 menjadi 'Other':")
print(result_df['subsequence'].value_counts()


Frekuensi subsequence setelah mengubah frekuensi < 2 menjadi 'Other':
subsequence
Other          51646
PEL               39
ADK               33
PDG               31
PEE               30
               ...  
PPG                2
SRQAQDLARSY        2
RKYN               2
AGFK               2
MGE                2
Name: count, Length: 7600, dtype: int64


In [ ]:
# prompt: cetak result_df yang memiliki subsequence other

result_df[result_df['subsequence'] == 'Other']

,Organism,subsequence,length_sub_seq,type_Encoded
0,Other,Other,-0.616144,0
1,Aphanizomenon flos-aquae (strain 2012/KM1/D3),Other,2.094104,0
3,Aphanizomenon flos-aquae (strain 2012/KM1/D3),Other,-0.435461,0
4,Aphanizomenon flos-aquae (strain 2012/KM1/D3),Other,3.900936,0
5,Aphanizomenon flos-aquae (strain 2012/KM1/D3),Other,0.467955,0
...,...,...,...,...
81983,Chlamydia pneumoniae (Chlamydophila pneumoniae),Other,0.467955,1
81984,Chlamydia pneumoniae (Chlamydophila pneumoniae),Other,-0.074094,1
81986,Rickettsia prowazekii (strain Madrid E),Other,0.467955,1
81987,Rickettsia prowazekii (strain Madrid E),Other,0.829322,1


In [ ]:
# # prompt: buatlah hashing encode yang menghasilkan integer untuk kolom Organism dan subsequence dengan nama yang sama dan drop kolomnya

# import pandas as pd
# import hashlib

# # Fungsi untuk melakukan hashing pada string
# def hash_string_to_int(string):
#   """Melakukan hashing pada string dan mengembalikan integer."""
#   if pd.isna(string):
#     return -1  # Atau nilai lain yang sesuai untuk NaN
#   # Gunakan SHA-256, lalu ambil 8 byte pertama dan konversi ke integer
#   hash_object = hashlib.sha256(str(string).encode())
#   return int.from_bytes(hash_object.digest()[:8], byteorder='big')

# # Terapkan hashing ke kolom 'Organism'
# result_df['Organism'] = result_df['Organism'].apply(hash_string_to_int)

# # Terapkan hashing ke kolom 'subsequence'
# result_df['subsequence'] = result_df['subsequence'].apply(hash_string_to_int)

# # Tampilkan hasil setelah hashing
# print("\nDataFrame setelah hashing:")
# print(result_df.head())

# # Drop kolom asli setelah hashing
# # result_df = result_df.drop(columns=['Organism', 'subsequence'])

# # Tampilkan info DataFrame setelah dropping kolom (opsional)
# # print("\nDataFrame info setelah drop kolom:")
# # print(result_df.info())

In [ ]:
# # prompt: buatlah frequency encode untuk kolom Organism dan subsequence dengan nama yang sama dan drop kolomnya

# # Frequency encode 'Organism'
# organism_freq = result_df['Organism'].value_counts()
# result_df['Organism'] = result_df['Organism'].map(organism_freq)

# # Frequency encode 'subsequence'
# subsequence_freq = result_df['subsequence'].value_counts()
# result_df['subsequence'] = result_df['subsequence'].map(subsequence_freq)

In [ ]:
# prompt: buatlah target encode dari Organism dengan target type

# Calculate target encoding for 'Organism' based on 'type_Encoded'
organism_target_encoding = result_df.groupby('Organism')['type_Encoded'].mean()

# Map the target encoding to the 'Organism' column in the DataFrame
result_df['Organism'] = result_df['Organism'].map(organism_target_encoding)

# prompt: buatlah target encode dari subsequence dengan target type

# Calculate target encoding for 'subsequence' based on 'type_Encoded'
subsequence_target_encoding = result_df.groupby('subsequence')['type_Encoded'].mean()

# Map the target encoding to the 'subsequence' column in the DataFrame
result_df['subsequence'] = result_df['subsequence'].map(subsequence_target_encoding)

# Display the updated DataFrame
result_df


,Organism,subsequence,length_sub_seq,type_Encoded
0,0.676195,0.585738,-0.616144,0
1,0.320000,0.585738,2.094104,0
2,0.320000,0.333333,-0.796827,0
3,0.320000,0.585738,-0.435461,0
4,0.320000,0.585738,3.900936,0
...,...,...,...,...
81984,0.500000,0.585738,-0.074094,1
81985,0.898305,0.818182,-0.796827,1
81986,0.898305,0.585738,0.467955,1
81987,0.898305,0.585738,0.829322,1


In [ ]:
# # Kelompokkan berdasarkan Organism_TargetEncoded dan hitung jumlah Organism unik
# grouped = result_df.groupby('Organism_TargetEncoded')['Organism'].nunique()

# # Temukan nilai Organism_TargetEncoded yang memiliki lebih dari satu Organism unik
# duplicates = grouped[grouped > 1]

# if not duplicates.empty:
#     print("Terdapat nilai Organism_TargetEncoded yang sama untuk Organism yang berbeda:")
#     for encoded_value, count in duplicates.items():
#         # Ambil Organism yang terkait dengan nilai Organism_TargetEncoded ini
#         related_Organisms = result_df[result_df['Organism_TargetEncoded'] == encoded_value]['Organism'].unique()
#         print(f"Organism_TargetEncoded: {encoded_value}, ditemukan pada Organism: {related_Organisms}")
# else:
#     print("Tidak ada nilai Organism_TargetEncoded yang sama untuk Organism yang berbeda.")

In [ ]:
# # Kelompokkan berdasarkan subsequence_TargetEncoded dan hitung jumlah subsequence unik
# grouped = result_df.groupby('subsequence_TargetEncoded')['subsequence'].nunique()

# # Temukan nilai subsequence_TargetEncoded yang memiliki lebih dari satu subsequence unik
# duplicates = grouped[grouped > 1]

# if not duplicates.empty:
#     print("Terdapat nilai subsequence_TargetEncoded yang sama untuk subsequence yang berbeda:")
#     for encoded_value, count in duplicates.items():
#         # Ambil subsequence yang terkait dengan nilai subsequence_TargetEncoded ini
#         related_subsequences = result_df[result_df['subsequence_TargetEncoded'] == encoded_value]['subsequence'].unique()
#         print(f"subsequence_TargetEncoded: {encoded_value}, ditemukan pada subsequence: {related_subsequences}")
# else:
#     print("Tidak ada nilai subsequence_TargetEncoded yang sama untuk subsequence yang berbeda.")

In [ ]:
# prompt: buatlah undersample dari type_encoded dari result_df

import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# Assuming 'type_Encoded' is your target variable
X = result_df.drop('type_Encoded', axis=1)
y = result_df['type_Encoded']

# Initialize RandomUnderSampler
rus = RandomUnderSampler(random_state=42)  # Set random_state for reproducibility

# Resample the data
X_resampled, y_resampled = rus.fit_resample(X, y)

# Create a new DataFrame with the resampled data
undersampled_df = pd.DataFrame(X_resampled, columns=X.columns)
undersampled_df['type_Encoded'] = y_resampled

# Now 'undersampled_df' contains your undersampled data
undersampled_df


,Organism,subsequence,length_sub_seq,type_Encoded
32506,0.700000,0.000000,0.287272,0
15527,0.729488,0.000000,1.010005,0
24660,0.676195,0.000000,0.648638,0
11598,0.703474,0.000000,-0.616144,0
6208,0.685124,0.000000,0.106589,0
...,...,...,...,...
43367,0.703474,1.500000,-0.616144,2
43368,0.653901,2.000000,-0.616144,2
43369,0.653901,0.833333,-0.796827,2
43370,0.898305,1.111111,-0.796827,2


In [ ]:
  # prompt: buatlah model cnn untuk undersampled_df

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Assuming 'undersampled_df' is your DataFrame and 'length_sub_seq', 'Organism_TargetEncoded', 'subsequence_TargetEncoded' are your features
# and 'type_Encoded' is your target variable.

# Prepare the data
X = undersampled_df[['length_sub_seq', 'Organism', 'subsequence']].values
y = undersampled_df['type_Encoded'].values

# Reshape X for 1D CNN (add a channel dimension)
# The second dimension should be large enough for MaxPooling1D to work without causing negative dimensions.
# Since you have 3 features, you can try setting the second dimension to 3.
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same')) # Add padding='same' to keep the output dimension the same as the input
model.add(MaxPooling1D(pool_size=2))  # You might need to adjust the pool_size based on the new shape of X
model.add(Flatten())
model.add(Dense(32, activation='relu'))  # Adjust units as needed
# model.add(Dropout(0.2))  # Add dropout for regularization
model.add(Dense(128, activation='relu'))  # Adjust units as needed
# model.add(Dropout(0.2))  # Add dropout for regularization
model.add(Dense(3, activation='softmax'))  # Output layer with softmax for multi-class classification


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)  # Adjust epochs and batch_size as needed

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2313/2313 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7909 - loss: 0.4777 - val_accuracy: 0.8716 - val_loss: 0.3006
Epoch 2/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8662 - loss: 0.2967 - val_accuracy: 0.8565 - val_loss: 0.3039
Epoch 3/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8666 - loss: 0.2964 - val_accuracy: 0.8696 - val_loss: 0.2991
Epoch 4/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8661 - loss: 0.2922 - val_accuracy: 0.8769 - val_loss: 0.2773
Epoch 5/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8676 - loss: 0.2996 - val_accuracy: 0.8706 - val_loss: 0.2868
Epoch 6/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8692 - loss: 0.2965 - val_accuracy: 0.8716 - val_loss: 0.2818
Epoch 7/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8733 - loss: 0.2849 - val_accuracy: 0.8721 - val_loss: 0.2787
Epoch 8/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8707 - loss: 0.2915 - val_accurac